In [1]:
import os
import h5py
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal
data_dir='ATL06/ATL06'

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


# make sure we're dealing with the most recent version of any code we're using
%load_ext autoreload
%autoreload 2

Analysis of clouds over outlet glaciers
Plot of ATL06 tracks after file transfers using aws

In [2]:
import numpy as np
import h5py


def ATL06_to_dict(filename, dataset_dict):
    """
        Read selected datasets from an ATL06 file

        Input arguments:
            filename: ATl06 file to read
            dataset_dict: A dictinary describing the fields to be read
                    keys give the group names to be read, 
                    entries are lists of datasets within the groups
        Output argument:
            D6: dictionary containing ATL06 data.  Each dataset in 
                dataset_dict has its own entry in D6.  Each dataset 
                in D6 contains a list of numpy arrays containing the 
                data
    """
    
    D6=[]
    pairs=[1, 2, 3]
    beams=['l','r']
    # open the HDF5 file
    with h5py.File(filename) as h5f:
        # loop over beam pairs
        for pair in pairs:
            # loop over beams
            for beam_ind, beam in enumerate(beams):
                # check if a beam exists, if not, skip it
                if '/gt%d%s/land_ice_segments' % (pair, beam) not in h5f:
                    continue
                # loop over the groups in the dataset dictionary
                temp={}
                for group in dataset_dict.keys():
                    for dataset in dataset_dict[group]:
                        DS='/gt%d%s/%s/%s' % (pair, beam, group, dataset)
                        # since a dataset may not exist in a file, we're going to try to read it, and if it doesn't work, we'll move on to the next:
                        try:
                            temp[dataset]=np.array(h5f[DS])
                            # some parameters have a _FillValue attribute.  If it exists, use it to identify bad values, and set them to np.NaN
                            if '_FillValue' in h5f[DS].attrs:
                                fill_value=h5f[DS].attrs['_FillValue']
                                temp[dataset][temp[dataset]==fill_value]=np.NaN
                        except KeyError as e:
                            pass
                if len(temp) > 0:
                    # it's sometimes convenient to have the beam and the pair as part of the output data structure: This is how we put them there.
                    temp['pair']=np.zeros_like(temp['h_li'])+pair
                    temp['beam']=np.zeros_like(temp['h_li'])+beam_ind
                    temp['filename']=filename
                    D6.append(temp)
    return D6

In [3]:
dataset_dict={'land_ice_segments':['h_li', 'delta_time','longitude','latitude'], 'land_ice_segments/ground_track':['x_atc']}

Trying to add a file to the directory and plot it

In [4]:
# NOTE: From now on, we'll be working in matplotlib's widget mode, which lets us zoom in on our plots.  
# This means that the figures won't be rendered in the notebook until you run them.
# That means no more spoiler plots (for now...)
%matplotlib widget


data_dir='/home/jovyan/ATL06/ATL06/'
D6=[]
pairs=[1, 2, 3]
beams=['l','r']

files=glob(data_dir+'/*.h5')
for file in files:
    this_name=os.path.basename(file)
    D6 += ATL06_to_dict(file, dataset_dict)
print("read %d beam/pair combinations" % (len(D6)))

# now plot the results:
plt.figure();
for Di in D6:
    plt.scatter(Di['longitude'], Di['latitude'], c=Di['h_li'], vmin=0, vmax=2000, linewidth=0)
plt.xlabel('longitude')
plt.ylabel('latitude')
plt.colorbar()

read 54 beam/pair combinations


FigureCanvasNbAgg()

In [ ]:
dataset_dict={'land_ice_segments':['latitude','longitude','h_li'], 'land_ice_segments/ground_track':['x_atc']}

%matplotlib widget
for file in files[0:5]:
    this_D6=ATL06_to_dict(file, dataset_dict)
    plt.figure()
    plt.plot(this_D6[1]['x_atc'], this_D6[1]['h_li'],'.')
    plt.title(this_D6[1]['filename'])

Code to read the data from Evan's folder


In [5]:
# NOTE: From now on, we'll be working in matplotlib's widget mode, which lets us zoom in on our plots.  
# This means that the figures won't be rendered in the notebook until you run them.
# That means no more spoiler plots (for now...)
%matplotlib widget


data_dir='/home/jovyan/xtrak/data/'
D6=[]
pairs=[1, 2, 3]
beams=['l','r']

files=glob(data_dir+'/*.h5')
for file in files:
    this_name=os.path.basename(file)
    D6 += ATL06_to_dict(file, dataset_dict)
print("read %d beam/pair combinations" % (len(D6)))

# now plot the results:
plt.figure();
for Di in D6:
    plt.scatter(Di['longitude'], Di['latitude'], c=Di['h_li'], vmin=0, vmax=2000, linewidth=0)
plt.xlabel('longitude')
plt.ylabel('latitude')
plt.colorbar()

read 324 beam/pair combinations


FigureCanvasNbAgg()

Now, lets look at few granules from a file

In [7]:
dataset_dict={'land_ice_segments':['latitude','longitude','h_li'], 'land_ice_segments/ground_track':['x_atc']}

%matplotlib widget
for file in files[9:15]:
    this_D6=ATL06_to_dict(file, dataset_dict)
    plt.figure()
    plt.plot(this_D6[1]['x_atc'], this_D6[1]['h_li'],'.')
    plt.title(this_D6[1]['filename'])

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()